In [1]:
from lusidtools.jupyter_tools import toggle_code

"""IRR Valuation

Attributes
----------
instruments
transactions
quotes
Recipe
IRR Valuation
"""

toggle_code("Toggle Docstring")

# IRR Calculations

In this example we demonstrate how Internal Rate of Return can be calculated using the GetValuation endpoint. We will set up a portfolio, create a simple instrument and upsert a number of related transactions. 

Once done, we will create a recipe for valuation and upsert quotes for the the simple instrument that we had created.

Using the valuation function we will illustrate the calculation of the IRR for the series of cashflows.

## Table of Contents:
- 1. [Imports](#1.-Imports)
- 2. [LUSID APIs](#2.-LUSID-APIs)
- 3. [Portfolio Creation](#3.-Portfolio-Creation)
- 4. [Instrument Creation](#4.-Instrument-Creation)
- 5. [Upsert Transactions](#5.-Upsert-Transactions)
- 6. [Valuation Recipe Creation](#6.-Valuation-Recipe-Creation)
- 7. [Upserting Market Data / Quotes Creation](#7.-Upserting-Market-Data-/-Quotes-Creation)
- 8. [Valuation with IRR](#8.-Valuation-with-IRR)
- 9. [Data Cleaning](#9.-Data-Cleaning)

# 1. Imports

In [ ]:
# Import LUSID libraries
import lusid 
import lusid.models as lm

from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame

# Import Libraries
from datetime import datetime, timedelta
from lusidtools.lpt.lpt import to_date
import pytz
import pandas as pd
import numpy as np
import json
import os
from lusidtools.cocoon.utilities import create_scope_id
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
    format_quotes_response,
    format_holdings_response,
)
from lusidtools.jupyter_tools import toggle_code
from lusidjam.refreshing_token import RefreshingToken

# Settings and utility functions to display objects and responses more clearly.
pd.set_option('float_format', '{:,.4f}'.format)

# Set the secrets path
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
        token=RefreshingToken(),
        api_secrets_filename = secrets_path,
        app_name="LusidJupyterNotebook")



# 2. LUSID APIs

Firstly, we initialize the LUSID APIs required for the notebook

In [ ]:
# Initiate the LUSID APIs required for the notebook
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
transaction_portfolio_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
portfolio_api = api_factory.build(lusid.api.PortfoliosApi)
quotes_api = api_factory.build(lusid.api.QuotesApi)
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)
aggregation_api = api_factory.build(lusid.AggregationApi)

# 3. Portfolio Creation

We proceed by creating a basic transaction portfolio:

In [ ]:
portfolio_scope= "Analytics-Examples1"
portfolio_code="IRR-Notebook-Equity1"
portfolio_name="IRR-Notebook-Equity1"
instrument_scope= "TestIRR1"
effective_at = datetime(2024, 5, 27, 0, 0, tzinfo=pytz.utc)

In [ ]:
def create_portfolio(scope, portfolio_code, name,instrument_scope):

    pf_df = pd.DataFrame(data=[
        {"portfolio_code": portfolio_code, "portfolio_name": name, "instrument_scope": instrument_scope},
    ])
    
    portfolio_mapping = {
        "required": {
            "code": "portfolio_code",
            "display_name": "portfolio_name",
            "base_currency": "$USD",
            "instrument_scopes": "instrument_scope"
        },
        "optional": {
            "created": f"${'01-01-2024'}"
        },
    }
    
    result = load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=pf_df,
        mapping_required=portfolio_mapping["required"],
        mapping_optional=portfolio_mapping["optional"],
        file_type="portfolios",
    )

    succ, failed = format_portfolios_response(result)
    display(pd.DataFrame(data=[{"success": len(succ), "failed": len(failed)}])) 

In [ ]:
create_portfolio(portfolio_scope, portfolio_code, portfolio_name, instrument_scope)

# 4. Instrument Creation

We create an equity instruments using lumi

In [ ]:
instr_df = pd.read_csv("IRR_instruments_upsert.csv")
display(instr_df)

In [ ]:
instrument_mapping = {
    "identifier_mapping": {
        "ClientInternal": "client_internal",
    },
    "required": {
        "name": "instrument_name"
    },
}

In [ ]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=portfolio_scope,
    data_frame=instr_df,
    mapping_required=instrument_mapping["required"],
    mapping_optional={},
    file_type="instruments",
    identifier_mapping=instrument_mapping["identifier_mapping"]
)

succ, failed, errors = format_instruments_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

# 5. Upsert Transactions

We can enter into a position in the equity, buy 100 @ $400 on 1st March for MSFT_41

In [ ]:
df1 = pd.read_csv("IRR_transactions.csv")
df1

In [ ]:
transaction_mapping = {
    "identifier_mapping": {"ClientInternal": "client_internal","LusidInstrumentId": "instrument_id"},
    "required": {
        "code": "portfolio_code",
        "transaction_id": "txn_id",
        "type": "txn_type",
        "transaction_price.price": "txn_price",
        "transaction_price.type": "$Price",
        "total_consideration.amount": "txn_consideration",
        "units": "txn_units",
        "transaction_date": "txn_trade_date",
        "total_consideration.currency": "portfolio_base_currency",
        "settlement_date": "txn_settle_date",
    },
    "optional": {},
    "properties": [],
}

In [ ]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=portfolio_scope,
    data_frame=df1,
    mapping_required=transaction_mapping["required"],
    mapping_optional=transaction_mapping["optional"],
    file_type="transactions",
    identifier_mapping=transaction_mapping["identifier_mapping"],
    property_columns=transaction_mapping["properties"],
)

succ, failed = format_transactions_response(result)
pd.DataFrame(
    data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]
)

# 6. Valuation Recipe Creation

Following the initial setup, we can see to configuring how LUSID will conduct valuation on the swap. This introduces the concept of recipes, which are a set of steps we specify to the valuation engine relating to market data and model specification.".

In [ ]:
recipe_code = "TestIRR_RecipeCode1"
recipe_scope = "Analytics-Examples1"
model_name = "SimpleStatic"

In [ ]:
# Create two different recipes depending on the AllowPartiallySuccessfulEvaluation option
def UpsertRecipe(recipe_scope,recipe_code,model_name):           
    try:
        configuration_recipe = lm.ConfigurationRecipe(
            scope=recipe_scope,
            code=recipe_code,
            market=lm.MarketContext(
                market_rules=[
                    lm.MarketDataKeyRule(
                        key="Quote.ClientInternal.*",
                        supplier="Lusid",
                        data_scope=recipe_scope,
                        quote_type="Price",
                        field="mid",
                        quote_interval="5D",
                    )
                ]
            ),
            pricing=lm.PricingContext(
                model_rules=[
                    lm.VendorModelRule(
                        supplier = "Lusid",
                        model_name = model_name,
                        instrument_type = "Equity",
                        parameters = "{}",
                    )
                 ],             
            )
        )
    
        upsert_configuration_recipe_response =  configuration_recipe_api.upsert_configuration_recipe(
                upsert_recipe_request=lm.UpsertRecipeRequest(
                    configuration_recipe=configuration_recipe
                )
            )
        
        print (f"Recipe {recipe_code} Upserted Successfully!")

    except lusid.ApiException as e:
        print(f"Recipie Creation Failed!")
        print(json.loads(e.body))
        

In [ ]:
UpsertRecipe(recipe_scope,recipe_code,model_name)

# 7. Upserting Market Data / Quotes Creation
We will be upserting quotes for the equity upserted earlier.

In [ ]:
#For first instrument
equity_prices = pd.DataFrame({
    'date' :["2024-03-01", "2024-03-27", "2024-04-01","2024-04-27", "2024-05-01", "2024-05-27"],
    'price' : [410, 420, 420, 430,430,440]
})
equity_prices.insert(0, 'ClientInternal', 'MSFT_41')
equity_prices.insert(3, 'currency', 'USD')

equity_prices

In [ ]:
quotes_mapping = {
    "quote_id.effective_at": "date",
    "quote_id.quote_series_id.provider": "$Lusid",
    "quote_id.quote_series_id.quote_type": "$Price",
    "quote_id.quote_series_id.instrument_id_type": "$ClientInternal",
    "quote_id.quote_series_id.instrument_id": "ClientInternal",
    "metric_value.unit": "currency",
    "metric_value.value": "price",
    "quote_id.quote_series_id.field": "$mid",
    
}

 
result = load_from_data_frame(
    api_factory = api_factory,
    scope=recipe_scope,
    data_frame=equity_prices,
    mapping_required=quotes_mapping,
    mapping_optional={},
    file_type="quotes"
)



succ, failed, errors = format_quotes_response(result)
display(pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]))

# 8. Valuation with IRR

In [ ]:
#Function to get valuation
def get_valuation(date: datetime,  portfolio_scope: str, portfolio_code: str, recipe_scope: str, recipe_code: str, metrics: list, groupBy: str=["Instrument/default/Name"]) -> pd.DataFrame:    
    
    try:
        valuation_request = lm.ValuationRequest(
            recipe_id=lm.ResourceId(
                scope=recipe_scope,
                code=recipe_code
            ),
            metrics=metrics,
            group_by=groupBy,
            portfolio_entity_ids=[
                lm.PortfolioEntityId(scope=portfolio_scope, code=portfolio_code)
            ],
            valuation_schedule=lm.ValuationSchedule(effective_at=date.isoformat()),
        )
    
        val_response = aggregation_api.get_valuation(valuation_request=valuation_request)
        val_data = val_response.data
        vals_df = pd.DataFrame(val_data)
        
        return vals_df
    
    except lusid.ApiException as e:
        print(json.loads(e.body)["errorDetails"][0]["id"])

In [ ]:
# Set the metrics to be requested from valuation
metrics = [
    lm.AggregateSpec("Instrument/default/Name", "Value"),
    lm.AggregateSpec("Instrument/default/ClientInternal", "Value"),
    lm.AggregateSpec("Valuation/PV", "Value"),
    lm.AggregateSpec("ProfitAndLoss/PortfolioInternalRateOfReturn", "Value", {"Window" : "MTD"}),
    lm.AggregateSpec("Holding/default/Units", "Value")
]

In [ ]:
df = get_valuation(effective_at, portfolio_scope,portfolio_code,recipe_scope,recipe_code,metrics,["Instrument/default/Name"])
df

# 8.1 IRR Explained
We get a Portfolio IRR of 38%, to validate this, we first confirm the valuation at the start of the Month

In [ ]:

df_start = get_valuation(datetime(2024, 5, 1, 0, 0, tzinfo=pytz.utc), portfolio_scope,portfolio_code,recipe_scope,recipe_code,[ lm.AggregateSpec("Instrument/default/Name", "Value"), lm.AggregateSpec("Valuation/PV", "Value") ],["Portfolio/default/Name"])

df_start

We have a value of 43,000 on 1st May 2024 (which is expected as the stock was $430) and then a final value of 44,000 on 27th May 2024.

The IRR value of 38\% can be validated in Excel using XIRR(), note the inital value should be set to -43,000.

We can also confirm it here by showing that:-43,000 + 44,000 / (1+irr) ^ (26 / 365) = 0

In [ ]:
irr = df.iloc[0,3]
days = 27 - 1

divisor = (1+irr)**(days/365)

-43_000 + 44_000 / divisor

# 9. Data Cleaning
The following chunks of code help you clean data by deleting recipes, quotes, instruments and portfolio created during the above sample

(for quotes and instruments you have to specify the instrument individually and effective date for quotes must match the effective date at time of creation)

In [ ]:
'''
#Delete Recipe
try:
    delete_recipe = configuration_recipe_api.delete_configuration_recipe(
        scope=recipe_scope,
        code=recipe_code,
    )

    print(delete_recipe)

except lusid.ApiException as e:
    print(json.loads(e.body)["title"])

In [ ]:
'''
#Delete Quotes

#Have to run this for individual instruments by changing instrument_id

try:
    delete_quotes = quotes_api.delete_quotes(
        scope=recipe_scope,
        request_body={ 
            "request_1": lm.QuoteId(
                quote_series_id=lm.QuoteSeriesId(
                    provider='Lusid',   
                    quote_type='Price',
                    instrument_id_type= 'ClientInternal',
                    instrument_id= 'MSFT_41',
                    field='mid'
                ),
                effective_at="2024-03-01T00:00:00Z"
            ),
            "request_2": lm.QuoteId(
                quote_series_id=lm.QuoteSeriesId(
                    provider='Lusid',   
                    quote_type='Price',
                    instrument_id_type= 'ClientInternal',
                    instrument_id= 'MSFT_41',
                    field='mid'
                ),
                effective_at="2024-03-27T00:00:00Z"
            ),
            "request_3": lm.QuoteId(
                quote_series_id=lm.QuoteSeriesId(
                    provider='Lusid',   
                    quote_type='Price',
                    instrument_id_type= 'ClientInternal',
                    instrument_id= 'MSFT_41',
                    field='mid'
                ),
                effective_at="2024-04-01T00:00:00Z"
            ),
            "request_4": lm.QuoteId(
                quote_series_id=lm.QuoteSeriesId(
                    provider='Lusid',   
                    quote_type='Price',
                    instrument_id_type= 'ClientInternal',
                    instrument_id= 'MSFT_41',
                    field='mid'
                ),
                effective_at="2024-04-27T00:00:00Z"
            ),
            "request_5": lm.QuoteId(
                quote_series_id=lm.QuoteSeriesId(
                    provider='Lusid',   
                    quote_type='Price',
                    instrument_id_type= 'ClientInternal',
                    instrument_id= 'MSFT_41',
                    field='mid'
                ),
                effective_at="2024-05-01T00:00:00Z"
            ),
            "request_6": lm.QuoteId(
                quote_series_id=lm.QuoteSeriesId(
                    provider='Lusid',   
                    quote_type='Price',
                    instrument_id_type= 'ClientInternal',
                    instrument_id= 'MSFT_41',
                    field='mid'
                ),
                effective_at="2024-05-27T00:00:00Z"
            )
            
            
    }
    )
        
    print(delete_quotes)

except lusid.ApiException as e:
    print(json.loads(e.body)["title"])


In [ ]:
'''
#Delete Instruments

#Have to run this for individual instruments by changing identifier value

try:
    delete_instrument = instruments_api.delete_instrument(
        identifier_type="ClientInternal", identifier= 'MSFT_41'
    )

    print(delete_instrument)

except lusid.ApiException as e:
    print(json.loads(e.body)["title"])


In [ ]:
'''
#Delete Portfolio
try:
    delete_portfolio = portfolio_api.delete_portfolio(portfolio_scope, portfolio_code)
        
    print(delete_portfolio)

except lusid.ApiException as e:
    print(json.loads(e.body)["title"])
